In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [2]:
df_silverBreweries = (
    spark
    .read
    .option("basePath", f"../datalake/silver/breweries/")
    .parquet(f"../datalake/silver/breweries/*")
)

In [10]:
df_aggBreweries = (
    df_silverBreweries
    .groupBy(
        F.col('brewery_type'),
        F.col('country')
    )
    .count()
    .orderBy(
        F.col('count').desc(),
        F.col('brewery_type').desc(),
        F.col('country').desc()
    )
)

In [11]:
df_aggBreweries.show()

+------------+-------------+-----+
|brewery_type|      country|count|
+------------+-------------+-----+
|       micro|United States| 4160|
|     brewpub|United States| 2420|
|    planning|United States|  674|
|      closed|United States|  245|
|    regional|United States|  215|
|    contract|United States|  188|
|       large|United States|   74|
|  proprietor|United States|   69|
|     brewpub|  South Korea|   56|
|       micro|      Ireland|   49|
|       micro|      England|   40|
|         bar|    Singapore|   28|
|       micro|       Poland|   20|
|        nano|United States|   12|
|     taproom|      England|   11|
|     brewpub|       Poland|   11|
|       micro|       Sweden|   10|
|       large|      Austria|   10|
|     brewpub|      England|   10|
|     taproom|United States|    9|
+------------+-------------+-----+
only showing top 20 rows



In [12]:
(
    df_aggBreweries
    .coalesce(1)
    .write    
    .mode('overwrite')
    .parquet(f"../datalake/gold/aggBreweries/")
)